# Mapping 1

Now that we've been able to import our first type of spatial data and begin manipulating it using our familiar libraries, we want to put it on a map!  

We begin with the code we developed last time for accessing the redlining data for a single city as a geopandas object:

In [1]:
#| hide
from IPython.display import IFrame

In [2]:
import ibis
from ibis import _
con = ibis.duckdb.connect()

city = (con
    .read_geo("/vsicurl/https://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
    .filter(_.city == "New Haven")
    .execute()
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

We will be using `leafmap` to create interactive maps. There are a wide range of mapping packages that have been developed for python (e.g. bokeh, kepler, folium, pydeck, lonboard, pymaplibre-gl), most of which wrap around javascript mapping libraries designed for interactive web-based maps.  Just as the `ibis` package provides an interface to a host of different database engines that can be used as 'backends', leafmap wraps around most of the common mapping engines with a friendly and mostly consistent user interface.  In `ibis` we have focused on the `duckdb` backend as the most powerful, flexible and performant option for all our purposes.  With `leafmap`, we will likewise focus on the most recent and generally most powerful backend option, [`maplibregl`](https://maplibre.org/maplibre-gl-js/docs/) - an open source implementation of the powerful and widely used commericial javascript platform, [Mapbox](https://mapbox.com).   

In [3]:
import leafmap.maplibregl as leafmap


In [4]:
#| hide-output
m = leafmap.Map(style="positron")
m.add_gdf(city)
m

Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': 'https://tiles.openfree…

Because our map is an interactive javascript applet, putting `m` at the end of the a cell will allow it to display inline in Jupyter, but not in 'static' documentation such as the course website or a GitHub rendering of the notebook.  To share our map, we can export it to an HTML file.  This allows any web browser to view and interact with the map without having to install `jupyter` or `python`.  

In [5]:
m.to_html("../data/nh1.html", overwrite=True)

In [6]:
#| hide-input
# display map on course website by using an iframe to the output URL 
IFrame(src='https://espm-157.carlboettiger.info/build/nh1-f227d00042a353d1f2aa8004304f376f.html', width=700, height=400)

[rendered map fullscreen](../data/nh1.html)

## Map styling 

Leafmap allows us to add many _layers_ to our map from various data sources, and gives us a rich set of tools for styling the look and feel of most of those data layers.  The example below illustrates how we can use the data column called "fill", which corresponds to the redlining "grade" associated with each layer, 



In [8]:
m = leafmap.Map(style="positron")


paint = {
    "fill-color": ["get", "fill"], # color by the column called "fill"
    "fill-opacity": 0.8,
}

m.add_basemap("Esri.NatGeoWorldMap")
m.add_gdf(city, layer_type="fill", name="Redlining", paint=paint)

m.add_layer_control()
m.to_html("../data/nh2.html", overwrite=True)
m

Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': 'https://tiles.openfree…

[rendered map fullscreen](../data/nh2.html)

In [ ]:
#| hide-input
# display map on course website by using an iframe to the output URL 
IFrame(src='https://espm-157.carlboettiger.info/build/nh2-f227d00042a353d1f2aa8004304f376f.html', width=700, height=400)

In [ ]:
## list all basemaps known to leafmap
#list(leafmap.basemaps.keys())

### Performance

Visualizing data on an interactive map is a challenging technical task because of the potential size of the data: To show the user data while they pan and zoom around the map across scales from city to planet-wide can involve trillions of pixels that must be loaded, plotted and styled.  Clever software takes advantage of the users's current zoom and field of view to render only the data the user can currently see, loading more data 'on demand' as the user moves the map.  Careful preparation of data layers into "tiles" pre-computed at different "zooms", data compression, and data chunking can dramatically improve the map's ability to render massive datasets smoothly. 